## Functions used for CBG-level and general inference and error analysis

In [ ]:
import json
import geopandas as gpd
import glob
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pyproj
import rasterio
import rasterio.features
import shapely
from shapely.geometry import box, Polygon
from tqdm import tqdm

In [ ]:
tqdm.pandas()

In [ ]:
def load_data(shapefile_fp, oak_fp):
    # CBG shapefiles
    cbg = gpd.read_file(os.path.join(shapefile_fp, 'tl_2021_06_bg'))
    cbg_scc = cbg.loc[cbg['COUNTYFP'] == '085']
    
    # Get San Jose city SHP
    scc_cities = gpd.read_file(os.path.join(shapefile_fp, 'City_Limits'))
    sj_city = scc_cities.loc[scc_cities['NAME'] == 'SAN JOSE']
    
    # Get CBGs in SJ city
    cbg_scc = cbg_scc.to_crs(sj_city.crs)
    cbg_sj = gpd.clip(cbg_scc, sj_city)
    
    # Zoning data
    print('[INFO] Restricting zoning data to R-1, R-2 and R-M (ex. R-MH)')
    zoning = gpd.read_file(os.path.join(oak_fp, 'san_jose_suppl', 'san_jose_Zoning_Districts.geojson'))
    zoning = zoning[(zoning['ZONING'].str.contains('R-1')) | (zoning['ZONING'].str.contains('R-2')) |\
                    ((zoning['ZONING'].str.contains('R-M')) & (zoning['ZONING'] != 'R-MH'))]
    zoning = zoning.to_crs(cbg_sj.crs)
    
    return cbg_sj, zoning

In [ ]:
def get_footprint_gpd(mask, file_name, size, tif_fp):
    """
    Converts predictions from np.array to shapely polygons
    :param mask: (np.array) Footprint mask from which to generate the polygons
    :param file_name: (str) Name of the TIF file 
    :param size: (int) Resolution size [512, 1024]
    :return: (gpd.GeoDataFrame)
    """

    with rasterio.open(os.path.join(tif_fp, f'{file_name}.tif')) as ds:
        t = ds.transform
        
    # Check whether tile is non-square
    raster = rasterio.open(os.path.join(tif_fp, '{}.tif'.format(file_name)))
    width = abs(raster.bounds[0] - raster.bounds[2])
    height = abs(raster.bounds[1] - raster.bounds[3])

    # Adjust for resolution
    factor1, factor2 = size // 512, size // 512
    factor1 = factor1 / (width/height) if height >= width else factor1
    factor2 = factor2 / (height/height) if width > height else factor2

    # Vectorize
    shapes = rasterio.features.shapes(mask, connectivity=4)
    polygons = [shapely.geometry.Polygon(shape[0]["coordinates"][0]) for shape in shapes if shape[1] == 1]
    polygons = [shapely.affinity.affine_transform(geom, [t.a / factor1, t.b, t.d, t.e / factor2, t.xoff, t.yoff]) for geom
                in polygons]
    buildings = gpd.GeoDataFrame(geometry=polygons, crs='EPSG:26910')
    buildings = buildings.to_crs('EPSG:4326')
    return buildings

In [ ]:
def collect_inferences(inferences_dir, size, tif_fp, output_fp):
    inference_files = glob.glob(os.path.join(inferences_dir, '*.npy'))
    
    building_footprints = gpd.GeoDataFrame()
    tile_bounds = {}
    
    for inference_file in tqdm(inference_files):
        # Convert inference to gpd
        file_name = inference_file.split(os.path.sep)[-1].replace('.npy', '')
        mask = np.load(inference_file)
        tile_buildings = get_footprint_gpd(mask, file_name, size, tif_fp)
        tile_buildings['tile'] = inference_file
        building_footprints = pd.concat([building_footprints, tile_buildings])
        
        # Get tile bounds
        with rasterio.open(os.path.join(tif_fp, f'{file_name}.tif')) as inds:
            bounds = inds.bounds
            geom = box(*bounds)

        # prepare to convert TIF bounds to standard 4326
        wgs84 = pyproj.CRS('EPSG:26910') # LA is 11, SJ is 10
        utm = pyproj.CRS('EPSG:4326')
        project = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform

        # Convert
        utm_geom = shapely.ops.transform(project, geom)
        tile_bounds[file_name] = list(utm_geom.exterior.coords)

    # Save tile_bounds dict
    with open(os.path.join(output_fp, 'tile_bounds.json'), "w") as f:
        json.dump(tile_bounds, f)
        
    return building_footprints

In [ ]:
def assign_cbgs(building_footprints, cbg_sj):
    # Assign buildings to San Jose CBGs
    building_footprints = building_footprints.to_crs(cbg_sj.crs)
    building_footprints_cbg = gpd.sjoin(
        cbg_sj[['GEOID', 'geometry']], building_footprints, 
        how='right', predicate='contains')

    # Drop buildings outside of San Jose
    building_footprints_cbg = building_footprints_cbg.dropna(axis=0, subset=['GEOID'])
    return building_footprints_cbg

In [ ]:
def combine_separate_buildings(building_footprints, sj_crs, verbose=True):
    # Merge buildings to connect buildings on tile edges
    if verbose:
        print('[INFO] Processing: Merging buildings on tile edges and combining outputs from overlapping tiles')
    geoms = building_footprints.geometry.unary_union
    building_footprints = gpd.GeoDataFrame(geometry=[geoms], crs='EPSG:4326')
    building_footprints = building_footprints.explode(index_parts=True).reset_index(drop=True)

    # Separate buildings
    if verbose:
        print('[INFO] Processing: Separating buildings')
    building_footprints = building_footprints.to_crs(crs=3857)
    
    if verbose:
        building_footprints.geometry = building_footprints.geometry.progress_apply(lambda x: x.buffer(-2))
    else:
        building_footprints.geometry = building_footprints.geometry.buffer(-2)
    building_footprints = building_footprints.explode(index_parts=False)
    
    if verbose:
        building_footprints.geometry = building_footprints.geometry.progress_apply(lambda x: x.buffer(2))
    else:
        building_footprints.geometry = building_footprints.geometry.buffer(2)
    building_footprints = building_footprints.to_crs(sj_crs)
    
    return building_footprints

In [ ]:
def filter_zoning(building_footprints, zoning, verbose=True):
    assert building_footprints.crs == zoning.crs
    
    # Remove predictions on roads
    if verbose:
        print('[INFO] Processing: Removing predictions on roads')
    if len(building_footprints) > 0:
        building_footprints = gpd.clip(building_footprints, zoning)

    return building_footprints

In [ ]:
def compute_area(building_footprints, small_upper_threshold, large_upper_threshold, sj_crs):
    # Compute building area
    building_footprints = building_footprints.to_crs('EPSG:26910')
    building_footprints['area'] = building_footprints.area
    
    # Filter for buildings that are too small or too large
    building_footprints = building_footprints.loc[building_footprints['area'] > 15]
    if large_upper_threshold:
        building_footprints = building_footprints.loc[building_footprints['area'] < large_upper_threshold]

    # Define small/large buildings
    # add up sqft/m^2 for just small buildings, or for both large and small buildings
    building_footprints['small'] = building_footprints['area'] < small_upper_threshold
    building_footprints['large'] = (1 - building_footprints['small']).astype(bool)
    
    building_footprints = building_footprints.to_crs(sj_crs)
    return building_footprints

In [ ]:
def process_buildings(cbg_sj, zoning, inferences_dir, size, small_threshold, 
                      large_threshold, fp, data_type, tif_fp, output_fp, save_temp=True):

    if data_type == 'inference':
        # 1) Loop over each tile to convert the inference np.array to gpd
        collected_footprints_file = os.path.join(fp, 'temp', 'raw_building_inferences')
        if not os.path.exists(collected_footprints_file):
            print('[INFO] Collecting building footprints')
            building_footprints = collect_inferences(inferences_dir, size, tif_fp, output_fp)
            
            if not os.path.exists(os.path.join(fp, 'temp')):
                os.makedirs(os.path.join(fp, 'temp'))
            building_footprints.to_file(collected_footprints_file)
    elif data_type == 'osm':
        collected_footprints_file = os.path.join(OAK_FP, 'san_jose_suppl', 'san_jose_OSM_footprints_res.geojson')
    else:
        raise Exception('[ERROR] Check data type')
        
    # 2) Process CBGs individually
    final_footprints_file = os.path.join(fp, '{}_building_processed'.format(data_type))
    if not os.path.exists(final_footprints_file):
        print('[INFO] Loading building footprints')
        building_footprints = gpd.read_file(collected_footprints_file)
        assert building_footprints.crs == cbg_sj.crs
        
        print('[INFO] Number of buildings: {}'.format(len(building_footprints)))
        
        # Assign CBGs
        print('[INFO] Assigning buildings to CBGs') 
        building_footprints = assign_cbgs(building_footprints, cbg_sj)
        building_footprints_processed = gpd.GeoDataFrame()
        
        for cbg in tqdm(cbg_sj.GEOID.unique()):
            cbg_file = os.path.join(fp, 'temp', '{}_building_cbg_{}'.format(data_type, cbg))
            cbg_geom = cbg_sj.loc[cbg_sj.GEOID == cbg].iloc[0]['geometry']
            
            # Get CBG building footprints
            building_footprints_cbg = building_footprints.loc[building_footprints['GEOID'] == cbg].copy()
            if len(building_footprints_cbg) > 0:
                if not os.path.exists(cbg_file):
                    # Process CBG inferred footprints: combine & separate
                    if data_type == 'inference':
                        building_footprints_cbg = combine_separate_buildings(
                            building_footprints_cbg, cbg_sj.crs, False)
                    if len(building_footprints_cbg) == 0:
                        continue
                        
                    # Filter roads & residential areas
                    zoning_cbg = gpd.clip(zoning, cbg_geom)
                    building_footprints_cbg = filter_zoning(
                        building_footprints=building_footprints_cbg, zoning=zoning_cbg, verbose=False)
                    assert building_footprints_cbg.crs == cbg_sj.crs

                    # Save temporary file
                    if len(building_footprints_cbg) > 0:
                        building_footprints_cbg['GEOID'] = cbg
                        if save_temp:
                            building_footprints_cbg.to_file(cbg_file)

                else:
                    building_footprints_cbg = gpd.read_file(cbg_file)
                
                # Append
                assert building_footprints_cbg.crs == building_footprints.crs
                building_footprints_processed = pd.concat([building_footprints_processed, building_footprints_cbg])
        
        # Compute area and filter for buildings that are too small
        print('[INFO] Computing small and large building area')
        building_footprints_processed = compute_area(
            building_footprints_processed, small_threshold, large_threshold, cbg_sj.crs)
        assert building_footprints_processed.crs == cbg_sj.crs
        
        # Save
        print('[INFO] Saving final building footprints')
        building_footprints_processed.to_file(final_footprints_file)

    # Read in final file
    building_footprints = gpd.read_file(final_footprints_file)

    return building_footprints

In [ ]:
def aggregate_cbg(cbg_sj, zoning, building_footprints_infer, building_footprints_osm, output_fp):
    cbg_path = os.path.join(output_fp, 'cbg_aggregate')
    if not os.path.exists(cbg_path):
        print('[INFO] Aggregating building inferences and OSM')
        building_footprints_dict = {'inf': building_footprints_infer, 'osm': building_footprints_osm}
        cbg_dict = {}
        
        # Aggregate (building area, building count)
        for data_type in ['inf', 'osm']:
            cbg_footprints_data = building_footprints_dict[data_type].groupby(
                    ['GEOID', 'small', 'large'])[['area']].agg(['sum','count']).reset_index()

            cbg_footprints_data.columns = cbg_footprints_data.columns.to_flat_index()
            cbg_footprints_data = cbg_footprints_data.rename(
                columns={('GEOID', ''): 'GEOID', ('small', ''): 'small',  
                         ('large', ''): 'large', ('area', 'sum'): 'b_a_{}'.format(data_type),
                   ('area', 'count'): 'b_n_{}'.format(data_type)})
            cbg_dict[data_type] = cbg_footprints_data
        
        cbg_footprints = pd.merge(
            cbg_dict['inf'], cbg_dict['osm'], how='left',
            on=['GEOID', 'small', 'large'], validate='one_to_one')
        
        # Add back geometry data to CBGs
        cbg_footprints = pd.merge(
            cbg_footprints, cbg_sj[['GEOID', 'geometry']], how='left', 
            on='GEOID', validate='many_to_one')
        cbg_footprints = gpd.GeoDataFrame(cbg_footprints)

        # Compute total CBG area
        cbg_footprints = cbg_footprints.to_crs('EPSG:26910')
        cbg_footprints['c_a_total'] = cbg_footprints.area
        cbg_footprints = cbg_footprints.to_crs('EPSG:4326')
        
        # Compute residential CBG area
        cbg_footprints_res = cbg_footprints.copy()
        cbg_footprints_res = cbg_footprints_res.loc[cbg_footprints_res.small == 1]

        assert zoning.crs == cbg_footprints_res.crs
        cbg_footprints_res = gpd.clip(cbg_footprints_res, zoning)
        
        cbg_footprints_res = cbg_footprints_res.to_crs('EPSG:26910')
        cbg_footprints_res['c_a_res'] = cbg_footprints_res.area
        cbg_footprints_res = cbg_footprints_res.to_crs('EPSG:4326')

        # Add residential area to main database
        cbg_footprints = cbg_footprints.merge(
            cbg_footprints_res[['GEOID', 'c_a_res']], on='GEOID', validate='many_to_one')
        
        # Compute % area
        for data_type in ['inf', 'osm']:
            cbg_footprints['b_ap_{}_t'.format(data_type)] = cbg_footprints['b_a_{}'.format(data_type)] / cbg_footprints['c_a_total'] * 100
            cbg_footprints['b_ap_{}_r'.format(data_type)] = cbg_footprints['b_a_{}'.format(data_type)] / cbg_footprints['c_a_res'] * 100
        
        # Save
        print('[INFO] Saving CBG file')
        cbg_footprints.to_file(cbg_path)

    cbg_footprints = gpd.read_file(cbg_path)
    return cbg_footprints

In [ ]:
def get_bounds(tile_bounds_dict, file_name):
    bounds = tile_bounds_dict[file_name]
    
    # Build polygon
    tile_poly = box(bounds[0][0], bounds[0][1], bounds[2][0], bounds[2][1])
    return tile_poly